# Imports

In [1]:
from scheduling_funtions import *
from constraints import *
from data import *
from model import *

# Solving the Model

In [2]:
NONE = 0
LOW = low_priority(len(shifts), len(weekends), len(staff))
MID = medium_priority(len(shifts), len(weekends), len(staff))
HIGH = high_priority(len(shifts), len(weekends), len(staff))
MAX = highest_priority(len(shifts), len(weekends), len(staff))

all_shifts_taken()
max_days_worked(staff, 7, 5, MAX)
# min_days_off_after_midnight(not_staff(midnight_staff), 2, 4, HIGH)
# midnight_physicians()
# no_midnights_within_six_months()
# max_midnights_in_a_row(not_staff(midnight_staff), 2, 1, MAX)
# on_call_shift_day_after()
# ft_physicians()
# no_late_shift_before_time_off()
# no_early_shifts_before_on_call()
# days_off_after_consecutive_shifts(staff, 2, 0, NONE)

# transitions_constraints(LOW, MID, HIGH, MAX)
# days_off_between_late_and_day_shifts(staff, 0, 3, HIGH)
# days_off_between_late_and_afternoon_shifts(staff, 0, 2, HIGH)
# late_shifts_in_a_row(staff, 3, 7, MAX)
# late_shifts_in_weeks(staff, 0, 0, NONE, 14, 5, MAX)
# avoid_consecutive_ft_shifts(not_staff(ft_staff), 2, 1, MID)

equalize_weekends(staff, triangle_costs(len(shifts), len(weekends), len(staff)), 4)
equalize_weekdays(staff, triangle_costs(len(shifts), len(weekdays), len(staff)), 1)


In [3]:
import time

start = time.time()

model.Minimize(
        sum(obj_bool_vars[i] * obj_bool_coeffs[i]
            for i in range(len(obj_bool_vars))) +
            sum(obj_int_vars[i] * obj_int_coeffs[i]
            for i in range(len(obj_int_vars))))

solver.parameters.max_time_in_seconds = 240.0
status = solver.Solve(model)

print(solver.StatusName(status))

for d in days:
    print('Day', d)
    for s in shifts:
        for m in staff:
            if solver.Value(works[(m, d, s)]) == 1:
              print(staff_list[m], 'works shift', shift_list[s])
    print()

end = time.time()
print(f"time elapsed: {end - start}")

UNKNOWN
Day 0


IndexError: list index out of range

# Tests

In [5]:
days_worked_results = {}
midnights_worked_results = {}
lates_worked_results = {}
day_shifts_worked_results = {}
shift_results = {}
ft_worked_results = {}
for m in staff:
    days_worked_results[m] = [solver.Value(days_assigned[(m, d)]) for d in days]
    midnights_worked_results[m] = [solver.Value(midnight_shifts_assigned[(m, d)]) for d in days]
    lates_worked_results[m] = [solver.Value(late_shifts_assigned[(m, d)]) for d in days]
    day_shifts_worked_results[m] = [solver.Value(day_shifts_assigned[(m, d)]) for d in days]
    ft_worked_results[m] = [solver.Value(ft_shifts_assigned[(m, d)]) for d in days]
    for d in days:
        shift_results[(m, d)] = [solver.Value(works[(m, d, s)]) for s in shifts]

## Hard Constraints

In [6]:
# No two doctors in the same shift on the same day
# All shifts should be taken by doctors
for d in days:
    num_staff_per_shift = zip(*[shift_results[(m, d)] for m in staff])
    assert(list(sum(column) for column in num_staff_per_shift) == [1] * len(shifts))
  
#No two shifts same day
for key, result in shift_results.items():
    assert(sum(result) <= 1)

# Maximum work 7 consecutive days
for key, result in days_worked_results.items():
    assert(not detect_pattern(result, "11111111"))

# # 2 days off after last midnight (except on call shift).
for key in midnights_worked_results.keys() and days_worked_results.keys():
    if midnight_only[key]:
         continue
    for idx in days[:-3]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            assert(sum(days_worked_results[key][idx+1:idx+3]) == 0)

# Certain staff work midnights only
for key, result in shift_results.items():
    if (midnight_only[key[0]] == 1):
        assert(result == [0,0,0,0,0,0,0,0,0,0,1] or result == [0] * len(shifts))

# No midnights for staff in their first 6 months (need way to indicate when physician is in first 6 months of practice)
for key, result in midnights_worked_results.items():
    if (first_six_month_only[key]):
        assert(sum(result) == 0)

# Maximum 2 midnights in a row (except for several physicians who only work midnights)
for key, result in midnights_worked_results.items():
    if midnight_only[key]:
         continue
    if (midnight_only[key] == 0):
        assert(not detect_pattern(result, "111"))

# On call shift - day after rules

# Certain physicians work only FT shift (0730,1530 shift)
for key, result in shift_results.items():
    if (staff_list[key[0]] == 1):
        assert(result == [0,1,0,0,0,0,0,0,0,0,0] or result == [0,0,0,0,0,1,0,0,0,0,0] or result == [0] * len(shifts))

# No 2000, 2200, or midnight shift prior to day requested off

# Physicians can work the 0930 shifts or earlier prior to working on call. They can work starting no earlier than 11 the day after on call.

# 2 days off after 3 to 7 days of work in a row
for key, result in days_worked_results.items():
    assert(not detect_pattern(result, "11101"))

print("ALL HARD CONSTRAINTS SATISFIED")

ALL HARD CONSTRAINTS SATISFIED


## Soft Constraints

In [7]:
# General principle avoid shift times changing too much day to day
# Shifts should have same start time to 2.5 hours later compared to previous shift (the 2 hours later can be relaxed to 3,4 perhaps)
# No shifts that start more than 1.5 hours earlier than the shift on the previous day
penalized_transitions = []

for shift in list(permutations(shift_list, 2)):
    t1 = float(shift[0][0:2] + '.' + shift[0][2:4])
    t2 = float(shift[1][0:2] + '.' + shift[1][2:4])
    if t2 - t1 > 2.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 1))
    elif t2 - t1 < -1.5:
        penalized_transitions.append(((shift_list.index(shift[0]), shift_list.index(shift[1])), 3))


transitions = {elem[0]: elem[1] for elem in penalized_transitions}
jumps_forward_25 = 0
jumps_backwards_15 = 0
previous_shift = -1
for key, result in shift_results.items():
    if previous_shift == -1:
        try:
            previous_shift = result.index(1)
        except:
            previous_shift = -1
    else:
        try:
            ans = transitions[(previous_shift, result.index(1))]
            if ans == 1:
                jumps_forward_25 += 1
            elif ans == 3:
                jumps_backwards_15 += 1
            previous_shift = result.index(1)
        except:
            previous_shift = -1

    if key[1] == len(days) - 1:
        previous_shift = -1 

print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()

Number of times shifts jump forwards 2.5 hours: 62
Number of times shifts jump backwards 1.5 hours: 62



In [8]:
# 3 days off after midnight. 4 even better
two_days_after_midnight = 0
three_days_after_midnight = 0
four_days_after_midnight = 0
five_days_after_midnight = 0
for key in midnights_worked_results.keys() and days_worked_results.keys():
    for idx in days[:-4]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0 and sum(days_worked_results[key][idx+1:idx+4]) == 1:
                two_days_after_midnight += 1
    for idx in days[:-5]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0 and sum(days_worked_results[key][idx+1:idx+5]) == 1:
                three_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+5]) == 0 and sum(days_worked_results[key][idx+1:idx+6]) == 1:
                four_days_after_midnight += 1
    for idx in days[:-6]:
        if midnights_worked_results[key][idx] == 1 and midnights_worked_results[key][idx+1] == 0:
            if sum(days_worked_results[key][idx+1:idx+6]) == 0:
                five_days_after_midnight += 1
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()

Number of times an employee gets 2 days off after midnights: 2
Number of times an employee gets 3 days off after midnights: 17
Number of times an employee gets 4 days off after midnights: 5
Number of times an employee gets 5 days off after midnights: 2



In [9]:
#Max 1 midnight in a row
two_midnights_in_a_row = 0
for key, result in midnights_worked_results.items():
    two_midnights_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()

Number of times an employee works 2 midnights in a row: 2



In [10]:
# 3 days off when transitioning from late shift to day shift 
one_days_off_from_late_to_day = 0
two_days_off_from_late_to_day = 0
three_days_off_from_late_to_day = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_day += 1
    for idx, elem in enumerate(lates_worked_results[key][:-4]):
        if elem == 1 and day_shifts_worked_results[key][idx + 4]:
            if sum(days_worked_results[key][idx+1:idx+4]) == 0:
                three_days_off_from_late_to_day += 1
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()


Number of times there is a day shift within 1 days of a late shift: 13
Number of times there is a day shift within 2 days of a late shift: 3
Number of times there is a day shift within 3 days of a late shift: 0



In [11]:
# 2 days off when transitioning from late shift to afternoon shift (although this transition should be avoided) 
one_days_off_from_late_to_afternoon = 0
two_days_off_from_late_to_afternoon = 0
for key in day_shifts_worked_results.keys() and lates_worked_results.keys():
    for idx, elem in enumerate(lates_worked_results[key][:-2]):
        if elem == 1 and day_shifts_worked_results[key][idx + 2]:
            if sum(days_worked_results[key][idx+1:idx+2]) == 0:
                one_days_off_from_late_to_afternoon += 1
    for idx, elem in enumerate(lates_worked_results[key][:-3]):
        if elem == 1 and day_shifts_worked_results[key][idx + 3]:
            if sum(days_worked_results[key][idx+1:idx+3]) == 0:
                two_days_off_from_late_to_afternoon += 1
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()

Number of times there is a afternoon shift within 1 days of a late shift: 13
Number of times there is a afternoon shift within 2 days of a late shift: 3



In [12]:
# 3 late shifts in a row maximum - late shifts are 1800, 2000, 2200. The ability to set what a late shift is would be great
three_days_in_a_row_late = 0
four_days_in_a_row_late = 0
five_days_in_a_row_late = 0
for key, result in lates_worked_results.items():
    three_days_in_a_row_late += detect_pattern_soft(result, '111')
    four_days_in_a_row_late += detect_pattern_soft(result, '1111')
    five_days_in_a_row_late += detect_pattern_soft(result, '11111')
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()

Number of times an employee works 3 late days in a row: 1
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0



In [13]:
# 5 late shifts in two weeks maximum


In [14]:
# Work maximum of 5 days in a row
#Max 1 midnight in a row
five_days_in_a_row = 0
six_days_in_a_row = 0
seven_days_in_a_row = 0
for key, result in days_worked_results.items():
    five_days_in_a_row += detect_pattern_soft(result, '11111')
    six_days_in_a_row += detect_pattern_soft(result, '111111')
    seven_days_in_a_row += detect_pattern_soft(result, '1111111')
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()

Number of times an employee works 5 days in a row: 0
Number of times an employee works 6 days in a row: 0
Number of times an employee works 7 days in a row: 0



In [15]:
# Avoid FT shifts (0730,1530) on consecutive days
two_ft_days_in_a_row = 0
for key, result in ft_worked_results.items():
    if ft_only[key]:
        continue
    two_ft_days_in_a_row += detect_pattern_soft(result, '11')
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

Number of times an employee works 2 ft shifts in a row: 6



In [16]:
print("SUMMARY")
print()
print(f"Number of times shifts jump forwards 2.5 hours: {jumps_forward_25}")
print(f"Number of times shifts jump backwards 1.5 hours: {jumps_forward_25}")
print()
print(f"Number of times an employee gets 2 days off after midnights: {two_days_after_midnight}")
print(f"Number of times an employee gets 3 days off after midnights: {three_days_after_midnight}")
print(f"Number of times an employee gets 4 days off after midnights: {four_days_after_midnight}")
print(f"Number of times an employee gets 5 days off after midnights: {five_days_after_midnight}")
print()
print(f"Number of times an employee works 2 midnights in a row: {two_days_after_midnight}")
print()
print(f"Number of times there is a day shift within 1 days of a late shift: {one_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 2 days of a late shift: {two_days_off_from_late_to_day}")
print(f"Number of times there is a day shift within 3 days of a late shift: {three_days_off_from_late_to_day}")
print()
print(f"Number of times there is a afternoon shift within 1 days of a late shift: {one_days_off_from_late_to_afternoon}")
print(f"Number of times there is a afternoon shift within 2 days of a late shift: {two_days_off_from_late_to_afternoon}")
print()
print(f"Number of times an employee works 3 late days in a row: {three_days_in_a_row_late}")
print(f"Number of times an employee works 4 late days in a row: {four_days_in_a_row_late}")
print(f"Number of times an employee works 5 late days in a row: {five_days_in_a_row_late}")
print()
print(f"Number of times an employee works 5 days in a row: {five_days_in_a_row}")
print(f"Number of times an employee works 6 days in a row: {six_days_in_a_row}")
print(f"Number of times an employee works 7 days in a row: {seven_days_in_a_row}")
print()
print(f"Number of times an employee works 2 ft shifts in a row: {two_ft_days_in_a_row}")
print()

SUMMARY

Number of times shifts jump forwards 2.5 hours: 62
Number of times shifts jump backwards 1.5 hours: 62

Number of times an employee gets 2 days off after midnights: 2
Number of times an employee gets 3 days off after midnights: 17
Number of times an employee gets 4 days off after midnights: 5
Number of times an employee gets 5 days off after midnights: 2

Number of times an employee works 2 midnights in a row: 2

Number of times there is a day shift within 1 days of a late shift: 13
Number of times there is a day shift within 2 days of a late shift: 3
Number of times there is a day shift within 3 days of a late shift: 0

Number of times there is a afternoon shift within 1 days of a late shift: 13
Number of times there is a afternoon shift within 2 days of a late shift: 3

Number of times an employee works 3 late days in a row: 1
Number of times an employee works 4 late days in a row: 0
Number of times an employee works 5 late days in a row: 0

Number of times an employee works

## Goals

In [17]:
saturdays = list(days[::7])
distance = 0
for m in staff:
    value = sum([solver.Value(days_assigned[(m, d)]) for d in weekends])
    print(f"{staff_list[m]} works {value} weekends")
    distance += abs(triangle_costs(len(shifts), len(weekends), len(staff)) - value)

print(f"Distance days from target weekends: {distance}")

Olivia works 6 weekends
Emma works 2 weekends
Ava works 5 weekends
Charlotte works 5 weekends
Sophia works 5 weekends
Amelia works 4 weekends
Isabella works 3 weekends
Mia works 6 weekends
Evelyn works 7 weekends
Harper works 7 weekends
Camila works 5 weekends
Gianna works 5 weekends
Abigail works 7 weekends
Luna works 7 weekends
Ella works 6 weekends
Elizabeth works 7 weekends
Sofia works 7 weekends
Emily works 5 weekends
Avery works 7 weekends
Mila works 4 weekends
Distance days from target weekends: 24


In [18]:
distance = 0
for m in staff:
    value = sum([solver.Value(days_assigned[(m, d)]) for d in weekdays])
    print(f"{staff_list[m]} works {value} weekdays")
    distance += abs(triangle_costs(len(shifts), len(weekdays), len(staff)) - value)

print(f"Distance days from target weekdays: {distance}")

Olivia works 11 weekdays
Emma works 12 weekdays
Ava works 11 weekdays
Charlotte works 13 weekdays
Sophia works 10 weekdays
Amelia works 9 weekdays
Isabella works 12 weekdays
Mia works 10 weekdays
Evelyn works 12 weekdays
Harper works 13 weekdays
Camila works 10 weekdays
Gianna works 10 weekdays
Abigail works 11 weekdays
Luna works 11 weekdays
Ella works 14 weekdays
Elizabeth works 13 weekdays
Sofia works 13 weekdays
Emily works 12 weekdays
Avery works 13 weekdays
Mila works 11 weekdays
Distance days from target weekdays: 23
